In [1]:
import numpy as np
import cv2 as cv
from scipy import ndimage
import math

In [2]:
def highPassFilter(img):
    filteredImg = np.zeros(img.shape)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            prev_val = img[i][j-1] if j > 0 else 0
            next_val = img[i][j+1] if j < img.shape[1]-1 else 0
            next_next_val = img[i][j+2] if j < img.shape[1]-2 else 0 
            filteredImg[i][j] = prev_val - 3*img[i][j] + 3*next_val - next_next_val
    return filteredImg

def gaussianHighPass(img):
    low_pass = ndimage.gaussian_filter(img, sigma=3)
    high_pass = img - low_pass
    return high_pass

In [3]:
def quantize(img, q, t):
    rounded_img = np.array([[int(element)//q for element in row] for row in img]).astype('uint8')
    truncated_img = np.array([[min(t, max(-t, element)) for element in row] for row in rounded_img]).astype('uint8')
    return truncated_img

In [4]:
# Read video into a VideoCapture object
vc = cv.VideoCapture('videos/video_pro2.avi')
# Read tampered video
vc_tamp = cv.VideoCapture('videos/v_out_4-2-2015-23-58-0.avi')
# Extract number of frames
num_frames = int(vc.get(cv.CAP_PROP_FRAME_COUNT))

In [5]:
# Testing flow values
prev_frame_num = num_frames/20
curr_frame_num = num_frames/20

vc.set(cv.CAP_PROP_POS_FRAMES, curr_frame_num-1)
status, frame = vc.read()
curr_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

vc.set(cv.CAP_PROP_POS_FRAMES, prev_frame_num-1)
status, frame = vc.read()
prev_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

vc_tamp.set(cv.CAP_PROP_POS_FRAMES, prev_frame_num-1)
status, frame = vc_tamp.read()
curr_tamp_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

# Resize images to 512x512 dimension
dim = (512,512)
curr_frame = cv.resize(curr_frame, dim, interpolation=cv.INTER_AREA)
curr_tamp_frame = cv.resize(curr_tamp_frame, dim, interpolation=cv.INTER_AREA)

In [ ]:
# Calculate flow
flow = cv.calcOpticalFlowFarneback(prev_frame, curr_frame, None, pyr_scale = 0.5, levels = 5, winsize = 11, iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)

print(flow[...,0].max(), flow[...,1].max())

In [6]:
quantized_img = quantize(gaussianHighPass(curr_frame), q=3, t=2)
tamp_quant_img = quantize(gaussianHighPass(curr_tamp_frame), q=3, t=2)

In [ ]:
cv.imshow('tampered', quantized_img)
cv.waitKey(7500)
cv.destroyAllWindows()

In [7]:
# Get index corresponding to cooccurrence
def cooccurrenceIndex(arr):
    # print(arr)
    return (arr[0]+2)*125 + (arr[1]+2)*25 + (arr[2]+2)*5 + arr[3]+2

# Compute co-occurrences
def computeRowCooccurrence(img):
    cooccurrence = np.zeros((625,), int)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]-3):
            cooccurrence[cooccurrenceIndex(img[i][j:j+4])] += 1
    return cooccurrence

# Compute co-occurrences
def computeColCooccurrence(img):
    cooccurrence = np.zeros((625,), int)
    for i in range(img.shape[0]-3):
        for j in range(img.shape[1]):
            cooccurrence[cooccurrenceIndex(img[i:i+4,j])] += 1
    return cooccurrence

In [8]:
def computeMeanDescriptors(descriptors):
    mean = np.zeros((625,), float)
    for j in range(625):
        mean[j] = (descriptors[0][j] + descriptors[1][j])/2.0
    return mean

def computeVarianceDescriptors(descriptors):
    mean = computeMeanDescriptors(descriptors)
    dif1 = descriptors[0]-mean
    dif2 = descriptors[1]-mean
    variance_sum = np.array([val*val for val in dif1], float).sum() + np.array([val*val for val in dif2], float).sum()
    return variance_sum/2.0

def computeMahalDist(descriptor, mean, variance):
    dif = descriptor-mean
    var_sum = np.array([val*val for val in dif], float).sum()
    return math.sqrt(var_sum/variance)

In [13]:
# Construct heat map
heat_map = np.zeros((512, 512), dtype=np.uint8)
block_size = (128,128)

for row in [block_size[0]*i for i in range(512//block_size[0])]:
    for col in [block_size[1]*i for i in range(512//block_size[1])]:
        block = quantized_img[row:row+block_size[0], col:col+block_size[1]]
        tamp_block = tamp_quant_img[row:row+block_size[0], col:col+block_size[1]]
        row_coocc = computeRowCooccurrence(block)
        col_coocc = computeColCooccurrence(block)
        tamp_row_coocc = computeRowCooccurrence(tamp_block)
        tamp_col_coocc = computeColCooccurrence(tamp_block)

        mean_desc = computeMeanDescriptors([row_coocc, col_coocc])
        variance = computeVarianceDescriptors([row_coocc, col_coocc])
        tamp_mean_desc = computeMeanDescriptors([tamp_row_coocc, tamp_col_coocc])

        heat_map[row:row+block_size[0],col:col+block_size[1]] = 255-100*computeMahalDist(tamp_mean_desc, mean_desc, variance)

In [14]:
cv.imshow('Heat map', heat_map)
cv.waitKey(7500)
cv.destroyAllWindows()